In [4]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import spacy
from spacy_langdetect import LanguageDetector
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,LabelEncoder


In [5]:
ds = load_dataset("maharshipandya/spotify-tracks-dataset")

In [6]:
df = pd.DataFrame(ds['train'])

In [7]:
df.drop(columns=['Unnamed: 0', 'track_id'], inplace=True)

In [8]:
scalar = MinMaxScaler()
target = df['popularity']
df.drop(columns=['popularity'], inplace=True)

In [9]:
df['duration_ms'] = scalar.fit_transform(df[['duration_ms']]).flatten()

In [10]:
encoder = OneHotEncoder(sparse_output=False)
temp = pd.DataFrame(encoder.fit_transform(df[['explicit']]), columns=encoder.get_feature_names_out(['explicit']))
temp2 = pd.DataFrame(encoder.fit_transform(df[['track_genre']]), columns=encoder.get_feature_names_out(['track_genre']))


In [11]:
df = pd.concat([df,temp,temp2], axis=1)
df.drop(columns=['explicit', 'track_genre'], inplace=True)

In [12]:
temp = pd.DataFrame(encoder.fit_transform(df[['key']]), columns=encoder.get_feature_names_out(['key']))
df = pd.concat([df,temp], axis=1)
df.drop(columns=['key'], inplace=True)

In [13]:

df['loudness'] = scalar.fit_transform(df[['loudness']]).flatten()
temp = pd.DataFrame(encoder.fit_transform(df[['time_signature']]), columns=encoder.get_feature_names_out(['time_signature']))
df=pd.concat([df,temp], axis=1)
df.drop(columns=['time_signature'], inplace=True)

In [20]:
nlp = spacy.load("en_core_web_sm")
@spacy.Language.factory("language_detector")

def create_lang_detector(nlp, name):
    return LanguageDetector()

nlp.add_pipe('language_detector', last=True)

def detect(text):
    doc = nlp(text)
    return doc._.language['language']
df['album_lang'] = df['album_name'].apply(detect)

KeyboardInterrupt: 